![QuantConnect Logo](https://cdn.quantconnect.com/web/i/logo-small.png)
## Welcome to The QuantConnect Research Page
#### Refer to this page for documentation https://www.quantconnect.com/docs#Introduction-to-Jupyter
#### Contribute to this template file https://github.com/QuantConnect/Lean/blob/master/Jupyter/BasicQuantBookTemplate.ipynb

In [1]:
%matplotlib inline
# Imports
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Custom import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Data.Consolidators import QuoteBarConsolidator
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from QuantConnect.Indicators.CandlestickPatterns import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import pdb

In [37]:
class Environment:
    
    def __init__(self, cash):
        self.START_CASH = cash
        self.cash = cash
        self.investQuantity = 0.05 # invest for each order relative to the cash sum
        self.state = 0 # 0:observe, 1:long, -1:short
        self.actionSpace = {0: self.orderLong, 1: self.orderShort, 2: self.closeOrder, 3: self.observeMarket}
    
    def reset(self):
        self.cash = self.START_CASH
        self.state = 0
    
    def orderLong(self):
        invest = self.calcInvest()
        self.order(invest, 1)
    
    def orderShort(self):
        invest = self.calcInvest()
        self.order(invest, -1)
    
    def observeMarket(self):
        pass
    
    def order(self, invest, newState):
        self.cash -= invest
        self.state = newState
        
    def closeOrder(self):
        self.state = 0
        self.cash += self.getReturn
    
    def calcInvest(self):
        return self.investQuantity * self.cash

class Rewarder:
    # calculates the reward for the execution
    
class Sampler:
    # returns a random dataset given the boundaries defined in the __init__ method
    
    def __init__(self, resolution):
        if resolution < 2: raise ValueError("Only minute (2), hour (3) and daily (4) data")
        
        self.qb = QuantBook()
        self.forexList = ["EURUSD", "GBPUSD", "USDJPY", "USDCAD", "AUSUSD", "USDCHF", "NZDUSD", "EURGBP", "EURJPY", "AUDJPY", "GBPJPY", "EURCHF"]
        self.MINIMUM_YEAR = 2008
        self.MAXIMUM_YEAR = 2015
        self.resolution = resolution
        self.MAX_RANDOMIZED_SAMPLE_LENGTH = {2:1440, 3:24, 4:0} # depending on resolution
        self.NUM_SAMPLES = 720
        self.WARUMUP_BUFFER_SAMPLES = 50
        self.DAYS_MONTH = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
        self.rsi = None
        self.macd = None
        self.standardDeviation = None
        self.adx = None
        self.addTradingObjects()
        self.addIndicators()
        
    def addTradingObjects(self):
        for fxSymbol in self.forexList:
            self.qb.AddForex(fxSymbol)
        
    def addIndicators(self):
        self.rsi = RelativeStrengthIndex(14)
        self.macd = MovingAverageConvergenceDivergence(12, 26, 9)
        self.standardDeviation = StandardDeviation(26)
        self.adx = AverageDirectionalIndex("ADX", 18)
    
    def getSample(self, snapshotLength):
        forexSymbol = self.getRandomForexSymbol()
        datetimeEnd = self.getRandomDatetime()
        self.qb.SetStartDate(datetimeEnd)
        randomSampleLength = random.randint(0, self.MAX_RANDOMIZED_SAMPLE_LENGTH[self.resolution]) # randomize the sample length to not start at the same time for each sample
        sampleLength = self.NUM_SAMPLES + randomSampleLength
        dataframeLength = sampleLength + self.WARUMUP_BUFFER_SAMPLES
        
        sample = Dataset(sampleLength, snapshotLength)
        sample.setPrice( self.qb.History([forexSymbol], dataframeLength, self.resolution) )
        sample.setRsi( self.qb.Indicator(self.rsi, forexSymbol, dataframeLength, self.resolution) )
        sample.setAdx( self.qb.Indicator(self.adx, forexSymbol, dataframeLength, self.resolution) )
        sample.setMacd( self.qb.Indicator(self.macd, forexSymbol, dataframeLength, self.resolution), \
                       self.qb.Indicator(self.standardDeviation, forexSymbol, dataframeLength, self.resolution) )
        
        return sample
        
    def getRandomForexSymbol(self):
        nSymbols = len(self.forexList)
        randomForexNumber = random.randint(0, nSymbols-1)
        return self.forexList[randomForexNumber]
    
    def getRandomDatetime(self, noMaximumLimitation = False):
        datetimeNow = datetime.now()
        maximumYear = datetimeNow.year if noMaximumLimitation else self.MAXIMUM_YEAR            
        randomYear = random.randint(self.MINIMUM_YEAR, maximumYear)
        randomMonth = random.randint(1, 12) if randomYear != datetimeNow.year else random.randint(1, datetimeNow.month)
        nDaysMonth = self.DAYS_MONTH[randomMonth] if not (self.isLeapYear(randomYear) and randomMonth == 2) else 29
        randomDay = random.randint(1, nDaysMonth)
        
        randomDatetime = datetime(randomYear, randomMonth, randomDay)
        latestDatetime = datetimeNow
        return min(randomDatetime, latestDatetime)
    
    def getSamplesRangeTimedelta(self):
        resolutionMinutes = self.RESOLUTION_MINUTES[self.resolution]
        minimumMinutes = resolutionMinutes * (self.NUM_SAMPLES + self.WARUMUP_BUFFER_SAMPLES)
        return timedelta(minutes = minimumMinutes)
    
    @staticmethod
    def isLeapYear(year):
        return True if year % 4 == 0 else False

class Dataset:
    
    def __init__(self, numBars, numObtainBars):
        self.length = numBars # defines how many bars with the beginning of the last datapoint we'll ingest from the dataframes
        self.numObtainBars = numObtainBars # defines how many last bars we should return when the iterator is called
        self.priceAsk = []
        self.priceBid = []
        self.rsi = []
        self.macdLine = []
        self.macdHistogram = []
        self.adx = []
        self.iterPosition = 0
        
    def __iter__(self):
        self.iterPosition = self.numObtainBars - 1
        return self
    
    def __next__(self):
        self.iterPosition += 1
        if self.iterPosition > self.length: raise StopIteration 

        startPoint = self.iterPosition - self.numObtainBars
        endPoint = self.iterPosition
        snapshot = self.createSnapshot(startPoint, endPoint)
        return snapshot
    
    def createSnapshot(self, startPoint, endPoint):
        snapshot = {}
        snapshot["ask"] = self.priceAsk[startPoint:endPoint]
        snapshot["bid"] = self.priceBid[startPoint:endPoint]
        snapshot["rsi"] = self.rsi[startPoint:endPoint]
        snapshot["macdLine"] = self.macdLine[startPoint:endPoint]
        snapshot["macdHistogram"] = self.macdHistogram[startPoint:endPoint]
        snapshot["adx"] = self.adx[startPoint:endPoint]
        
        return snapshot
    
    def setPrice(self, priceDataframe):
        self.priceAsk = priceDataframe['askclose'].values[-self.length:]
        self.priceBid = priceDataframe['bidclose'].values[-self.length:]
    
    def setRsi(self, rsiDataframe):
        self.rsi = rsiDataframe['relativestrengthindex'].values[-self.length:]
    
    def setAdx(self, adxDataframe):
        self.adx = adxDataframe['averagedirectionalindex'].values[-self.length:]
    
    def setMacd(self, macdDataframe, stddevDataframe):
        macdLine = macdDataframe['movingaverageconvergencedivergence'].values[-self.length:]
        macdHistogram = macdDataframe['histogram'].values[-self.length:]
        standardDeviation = stddevDataframe['standarddeviation'].values[-self.length:]
        self.macdLine = [macd/stddev for macd, stddev in zip(macdLine, standardDeviation)] # norm the data due to comparison between different price levels
        self.macdHistogram = [histogram/stddev for histogram, stddev in zip(macdHistogram, standardDeviation)] # norm the data due to comparison between different price levels        

In [39]:
sampler = Sampler(Resolution.Hour)
sample = sampler.getSample(24)
i = 0
for smpl in sample:
    i+=1
    print(i)
    print(len(smpl["macdLine"]))

1
24
2
24
3
24
4
24
5
24
6
24
7
24
8
24
9
24
10
24
11
24
12
24
13
24
14
24
15
24
16
24
17
24
18
24
19
24
20
24
21
24
22
24
23
24
24
24
25
24
26
24
27
24
28
24
29
24
30
24
31
24
32
24
33
24
34
24
35
24
36
24
37
24
38
24
39
24
40
24
41
24
42
24
43
24
44
24
45
24
46
24
47
24
48
24
49
24
50
24
51
24
52
24
53
24
54
24
55
24
56
24
57
24
58
24
59
24
60
24
61
24
62
24
63
24
64
24
65
24
66
24
67
24
68
24
69
24
70
24
71
24
72
24
73
24
74
24
75
24
76
24
77
24
78
24
79
24
80
24
81
24
82
24
83
24
84
24
85
24
86
24
87
24
88
24
89
24
90
24
91
24
92
24
93
24
94
24
95
24
96
24
97
24
98
24
99
24
100
24
101
24
102
24
103
24
104
24
105
24
106
24
107
24
108
24
109
24
110
24
111
24
112
24
113
24
114
24
115
24
116
24
117
24
118
24
119
24
120
24
121
24
122
24
123
24
124
24
125
24
126
24
127
24
128
24
129
24
130
24
131
24
132
24
133
24
134
24
135
24
136
24
137
24
138
24
139
24
140
24
141
24
142
24
143
24
144
24
145
24
146
24
147
24
148
24
149
24
150
24
151
24
152
24
153
24
154
24
155
24
156
24
157
24
158
24
15

In [ ]:
pdb.pm()